## Usando o rENA no python

O exemplo abaixo mostra como utilizar a biblioteca do R rENA com a liguagem python.
Todas as bibliotecas do python podem ser usadas para mnipular e arrumar os dados para serem plotados.

Todas as funçoes da biblioteca podem ser chamadas usando a interface rENA, um outro detalhe ,e que os pontos (.) devem ser substituidos por underscore (_)
Exemplo:
    A função ena.plot() pode ser chamada usando rENA.ena_plot()
    A função ena.plot.points() pode ser chamada usando rENA.ena_plot_points().
    
### Bibliotecas necessárias:
    Devem ser instaladas as bibliotecas R rENA e data.table usando os comandos abaixo:
    install.packages("rENA")
    install.packages("data.table")
    
    No python devemos instalar a biblioteca rpy2 usando o seguinte comando:
    pip install rpy2
    
    

In [1]:
# -*- coding: UTF-8 -*-

import pandas as pd
import os
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()  # makes some conversions automatic

## Importação da biblioteca rENA do R
Suas funções estarão disponíveis na variável "rENA"

In [2]:
rENA = importr('rENA')

## A função robjects possibilita a utilização de funções do R dentro python

In [3]:
#funções do R preparadas para serem chamadas no python
pr = ro.r("print")
colMeans = ro.r("colMeans")
rmax = ro.r("max")
savePlot = ro.r("savePlot")

In [4]:
# Leitura dos dados a serem plotados
#rsdata = pd.read_csv("rsdata.csv")
# load your file
rsdata = pd.read_csv("rsdata.csv")

# Set "Browser"

R will want to display our plots using a browser, but in binder, we don't have a "browser." So, we'll set an environment variable to tell it to run our browser script instead.

In [5]:
os.environ["BROWSER"] = "/home/jovyan/browser.sh"

## Identificar colunas para usar no acumulador
Antes de executar a função ena_accumulate_data, precisamos primeiro identificar quais colunas de dados usar em nossas unidades (units), conversas (conversation) e códigos (codes). Há também um parâmetro opcional de metadados (meta), que é para dados específicos da unidade que desejamos realizar no processo de acumulação e nos manter associados às unidades identificadas.

In [6]:
units = rsdata[['Condition','UserName']]
conversation = rsdata[['Condition','GroupName']]
codes = rsdata[['Data','Technical.Constraints','Performance.Parameters','Client.and.Consultant.Requests','Design.Reasoning','Collaboration']]
meta = rsdata[["CONFIDENCE.Change","CONFIDENCE.Pre","CONFIDENCE.Post","C.Change"]]

## Executando a acumulação dos dados
Com os dados identificados e subconjuntos, a acumulação e a geração de conjuntos são bastante simples.

In [7]:
accum = rENA.ena_accumulate_data(units, conversation, codes, meta)

## Gerando o set (confjunto) ENA
A forma mais básica de um ENAset pode ser gerada passando o resultado da chamada da função ena_accumulate_data para ena_make_set.

In [8]:
set = rENA.ena_make_set(
  enadata=accum
)

## Plot Units In Each Group

In [9]:
points = (set.rx2('points'))

df_points1 = points.loc[points['Condition'] == "FirstGame"]
df_points2 = points.loc[points['Condition'] == "SecondGame"]
### Subset rotated points for the first condition
first_game_points = df_points1.drop(columns=['ENA_UNIT','Condition','UserName', 'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])
### Subset rotated points for the second condition
second_game_points = df_points2.drop(columns=['ENA_UNIT','Condition','UserName',  'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])



plot = rENA.ena_plot(set, scale_to ="network", title ="Groups of Units")
plot = rENA.ena_plot_points(plot, points = first_game_points, confidence_interval ="box", colors = ("blue"))
plot = rENA.ena_plot_points(plot, points = second_game_points, confidence_interval ="box", colors = ("red"))
pr(plot)

<rpy2.robjects.vectors.ListVector object at 0x7f09c18f9690> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18eaf00> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18db8c0> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18e80a0> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f09c18e87d0> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18e97d0> [RTYPES.VECSXP]

## Plotting Means

In [10]:
plot = rENA.ena_plot(set, scale_to = [-1, 0, 1], title ="Groups and Means")
plot = rENA.ena_plot_points(plot, points = first_game_points, confidence_interval ="box", colors = ("blue"))
plot = rENA.ena_plot_points(plot, points = second_game_points, confidence_interval ="box", colors = ("red"))
plot = rENA.ena_plot_group(plot, first_game_points, colors = ("red"), confidence_interval ="box")
plot = rENA.ena_plot_group(plot, second_game_points, colors =("blue"), confidence_interval ="box")
pr(plot)

<rpy2.robjects.vectors.ListVector object at 0x7f09c18988c0> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a7fa0> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a7d20> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a7870> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f09c18a7c80> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a78c0> [RTYPES.VECSXP]

## Plotting a Network

In [11]:
line_weights = (set.rx2('line.weights'))

### Subset lineweights for SecondGame and Calculate the colMeans
first_game_lineweights = line_weights.loc[points['Condition'] == "FirstGame"]
second_game_lineweights = line_weights.loc[points['Condition'] == "SecondGame"]

first_game_lineweights = first_game_lineweights.drop(columns=['ENA_UNIT','Condition','UserName', 'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])
### Subset rotated points for the second condition
second_game_lineweights = second_game_lineweights.drop(columns=['ENA_UNIT','Condition','UserName',  'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])

first_game_mean = colMeans(first_game_lineweights)
second_game_mean = colMeans(second_game_lineweights)

subtracted_mean = first_game_mean - second_game_mean

In [12]:
plot_first = rENA.ena_plot(set, title ="FirstGame")
plot_first = rENA.ena_plot_network(plot_first, network = first_game_mean)
pr(plot_first)

<rpy2.robjects.vectors.ListVector object at 0x7f0a0874deb0> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09e30a2730> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f0a0d218e10> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f0a08754190> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f0a08708140> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18b0460> [RTYPES.VECSXP]

In [13]:
plot_second = rENA.ena_plot(set, title ="SecondGame")
plot_second = rENA.ena_plot_network(plot_second, network = second_game_mean, colors = ("blue"))
pr(plot_second)

<rpy2.robjects.vectors.ListVector object at 0x7f09c18e24b0> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18e9730> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18ddfa0> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a2cd0> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f09c18a2d20> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a7e10> [RTYPES.VECSXP]

## Plot subtracted network only

In [14]:
plot_sub = rENA.ena_plot(set, title ="Subtracted")
plot_sub = rENA.ena_plot_network(plot_sub, network = subtracted_mean)
pr(plot_sub)

<rpy2.robjects.vectors.ListVector object at 0x7f09c18f7640> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18a6550> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c189fe10> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c189f3c0> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f09c18b2a50> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18b2b40> [RTYPES.VECSXP]

## Plot Everything Together

In [15]:
rotation = (set.rx2('rotation'))
nodes = rotation.rx2('nodes')

# Scale the nodes to match that of the network, for better viewing
first_game_points_max = rmax(first_game_points)
second_game_points_max = rmax(second_game_points)
if(first_game_points_max> second_game_points_max):
  point_max = first_game_points_max
else:
  point_max = second_game_points_max

nodes = nodes.drop(columns=['code'])
max_nodes = rmax(nodes)
with localconverter(ro.default_converter + pandas2ri.converter):
  first_game_points = ro.conversion.py2rpy(first_game_points)

first_game_scaled = first_game_points
second_game_scaled = second_game_points

plot = rENA.ena_plot(set, title ="Plot with Units and Network", font_family ="Times")
plot = rENA.ena_plot_points(plot, points = first_game_scaled, colors = ("red"))
plot = rENA.ena_plot_points(plot, points = second_game_scaled, colors = ("blue"))
plot = rENA.ena_plot_group(plot, point = first_game_scaled, colors =("red"), confidence_interval ="box")
plot = rENA.ena_plot_group(plot, point = second_game_scaled, colors =("blue"), confidence_interval ="box")
plot = rENA.ena_plot_network(plot, network = subtracted_mean)
pr(plot)

<rpy2.robjects.vectors.ListVector object at 0x7f09c18c10f0> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18bf190> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18bd820> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18bd140> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7f09e89fc870> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7f09c18baaf0> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f09c18cb0f0> [RTYPES.VECSXP]